In [2]:
import json
import glob
import os

In [3]:
caveat_files_dir = '/media/thien/Data Drive1/api_12_caveat_sentences/'
caveats_dict = {}

files = glob.glob(caveat_files_dir + '*.json')
for file in files:
    with open(file) as f:
        arr = json.load(f)
        name = os.path.basename(file)
        
        caveats_dict[name] = arr

In [7]:
print(len(caveats_dict))

4212
